In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import os

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(dictres):
#extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    for i in range(0,len(allSlices)):
        df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
    return df_sum

In [5]:
def slicesParser_halfpeakstime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

In [6]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [7]:
def slicesParser_min_peaksI(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

    return df_sum

In [8]:
def slicesParser_peaktime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']

    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

    return df_sum

In [13]:
# Calcula el alternance ratio

AR_list = []
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
jsons_list = os.listdir(path)
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    slices = dic['slices']
    i = 0
    df = pd.DataFrame(columns=['transient1','transient2','transient_max'])
    print (file)
    for i in range(0,33):
        try:
            amps = slices[i]['amplitudes'][0:2]
            amps.append(max(amps))
            i += 1
            df.loc[len(df)] = amps
        except IndexError:
            pass
    df.to_csv(path + file + 'max_amp.csv', sep='\t')
print('Done')

FileNotFoundError: [Errno 2] No such file or directory: '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'

In [20]:
# Guarda en tablas separadas los datos de cada json por parámetro

param_list = ['max_peaks_positions','min_peaks_positions','max_peaks_intensities','min_peaks_intensities','amplitudes','times_to_peaks','times_to_half_peaks']

for param in param_list:
    os.makedirs(path2 + param + '/')
    path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
    path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
    jsons_list = os.listdir(path)
    for file in jsons_list:
        path_file = path + file
        dic = jasonReader(path_file)
        slices = dic['slices']
        i = 0
        df = pd.DataFrame(columns=['transient1','transient2'])
        print (file)
        for i in range(0,33):
            try:
                amps = slices[i][param][0:2]
                i += 1
                df.loc[len(df)] = amps
            except IndexError:
                pass
        df.to_csv(path2 + param + '/' + file + param + '.csv', sep='\t')
    print('Done')

050320c8_analysis_result_8b000.json
050320c7_analysis_result_7d001.json
050320c6_analysis_result_6d001.json
220119VK1uM_analysis_result_vk2a000.json
010719ctrol8_analysis_result_ctrol8b0.json
070119Ctrol6_analysis_result_c6b000.json
050320c4_analysis_result_4d001.json
171019c1sel_analysis_result_c1a000.json
010819c1_analysis_result_c1a000.json
031019c1_analysis_result_c1e000.json
050320c3_analysis_result_3a001.json
050320c2_analysis_result_2d000.json
171019vk3_analysis_result_vk3a000.json
230819VK1_analysis_result_vk1d000.json
230819Dan2_analysis_result_D2d000.json
010719VK7_analysis_result_vk7b000.json
031019c3_analysis_result_c3b000.json
170719c1_analysis_result_c1b000.json
070119Ctrol7_analysis_result_c7a000.json
311019vk4_analysis_result_vk4a000.json
050320c5_analysis_result_5d001.json
230819VK1_analysis_result_vk1a000.json
170719Dan8_analysis_result_dan8b000.json
050320c1_analysis_result.json
031019c3_analysis_result_c3f000.json
171019c6sel_analysis_result_c6d000.json
170719c1_ana

In [41]:
# Guarda en una tabla los datos de la célula entera

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    pairs = {k: dic[k] for k in list(dic)[1:]}
    keys = list(pairs.keys())
    df = pd.DataFrame()
    for label in range(0,len(pairs)):
        df[keys[label]] = pairs[keys[label]][0:2]
    df = df.T
    df.columns = ['Transient1', 'Transient2']
    df.to_csv(path2 + 'whole_cell/' + file + '.csv', sep='\t')
print('Done')

Done


In [ ]:
# Cálculo del TAU67%



In [ ]:
# Recalcula la amplitud normalizando por el baseline de 1Hz

TAU = pd.DataFrame()
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
path2 = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    for peak in range  (0, cantidad_total_picos):
        amp50 = (intensidades_maximas ['Pico_' + str(peak)] + intensidades_minimos ['Pico_' + str(peak)])/2
        amp50 = list (amp50.astype(np.int))
        lista_ttp50 = []
        for i in range (0, len (amp50)):
            lista_up = np.asarray (list_img_row [i] [tiempos_minimos ['Pico_' + str(peak)][i] : tiempos_maximos ['Pico_' + str(peak)][i]])
            ttp50 = (np.abs(lista_up - amp50[i])).argmin()
            lista_ttp50.append (ttp50)
        tiempo_50pico ['Pico_' + str(peak)] = lista_ttp50